<a href="https://colab.research.google.com/github/navidTerraNova/crossword-puzzle-clue-generation-using-LLMs/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PACKAGES INSTALLATION**

In [ ]:
!pip install rouge_score nltk transformers transformers[sentencepiece] datasets

# **MOUNTING GOOGLE DRIVE**

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")

# **LOADING DATAFRAME**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DataSet/nytcrosswords.csv", encoding='latin1')

## **NAN VALUE CHECK**

In [ ]:
counter = 0
for i in range(len(df["Word"])):
  if df["Word"][i] == "NAN":
    counter = counter + 1
    print("found None Value")
    print(df["Word"][i])
    print(i)

print(counter)

In [ ]:
counter = 0
for i in range(len(df["Word"])):
  if df["Word"][i] != df["Word"][i] :
    counter = counter + 1
    print("found None Value")
    print(df["Word"][i])
    print(i)

print(counter)

# **DATA PREPROCESS**

In [ ]:
df = df.drop(["Date"], axis = 1)
df["Word"] = df["Word"].str.lower()
df = df[~df["Clue"].str.contains("___")]
df = df.dropna()
df = df[~df["Word"].str.contains("nan")]
df = df.drop_duplicates()
df = df.reset_index(drop = True)

In [ ]:
df = df.sample(frac=1, random_state=2)

In [ ]:
df = df[:500000]

In [ ]:
df.to_csv('NYT-Crossword-Cleaned.csv', index = False)

# **LOADING DATASET**

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files = "/content/NYT-Crossword-Cleaned.csv")

## **TRAIN_VALID_TEST**

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.02, shuffle=False)

In [ ]:
datasets_train_validation = dataset["train"].train_test_split(test_size=5000)

dataset["train"] = datasets_train_validation["train"]
dataset["validation"] = datasets_train_validation["test"]

In [ ]:
dataset["train"] = dataset["train"].shuffle().select(range(50000))
dataset["validation"] = dataset["validation"].shuffle().select(range(2000))

# **TOKENIZE**

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def preprocess_function(examples):

  model_inputs = tokenizer(examples["Word"], max_length = 16, truncation=True, padding = True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["Clue"], max_length = 32, truncation=True, padding = True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched = True)

In [ ]:
print(tokenized_datasets["train"]["Clue"][2],tokenized_datasets["train"]["labels"][2])

In [ ]:
tokenized_datasets

# **SETTING TRAINING ARGUMENTS**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
batch_size = 32
model_name = "FTT5-(500000)"
model_dir = f"drive/MyDrive/Models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=2000,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

# **METRIC SETUP**

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# **TRAINER API SETUP**

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:

# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# **PLOT**

In [ ]:
# Start TensorBoard before training to monitor it in progress
%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

In [ ]:
%reload_ext tensorboard

In [ ]:
trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/8000 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
8200,0.852000,0.894035,4.312800,0.435700,4.224700,4.241800,7.707300
8400,0.841600,0.892978,4.370100,0.457900,4.292700,4.306300,7.315300
8600,0.846200,0.891503,4.374000,0.433000,4.294200,4.314800,7.378000
8800,0.861800,0.890067,4.514900,0.430700,4.423200,4.444000,7.885200
9000,0.836900,0.889520,4.689900,0.504400,4.600400,4.622600,8.012400
9200,0.823900,0.888058,4.589800,0.477700,4.499600,4.514500,7.677500
9400,0.844300,0.887016,4.669100,0.501600,4.579700,4.592300,7.835400
9600,0.847200,0.886131,4.733600,0.488700,4.647400,4.668200,8.027900
9800,0.838900,0.885407,4.669000,0.487900,4.577600,4.594300,7.779700
10000,0.826300,0.884348,4.700600,0.497600,4.607400,4.625800,7.760600


KeyboardInterrupt: ignored

# **INFERENCE**

In [ ]:
model_name = "FTT5-(300000)/checkpoint-35000"
model_dir = f"drive/MyDrive/Models/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 16

In [ ]:
for i in range(40):
  inputs = "history"

  inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
  output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=1, max_length=15)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

  print(predicted_title)